In [1]:
import torch
import torch.nn as nn
import pandas as pd
#import math
import matplotlib.pyplot as plt
from tqdm import tqdm

import numpy as np
import torch.nn.functional as F
#from torchvision.utils import make_grid, save_image
import torch.distributions as dist
import os

from torch.utils.data import Dataset, DataLoader

import nltk
import re
from nltk.tokenize import word_tokenize
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
import torch.optim as optim

In [ ]:
df = pd.read_csv("/home/vcourrier/these_code/mva_project/data/IMDB Dataset.csv")
print(df.head(), "\n")

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive 



In [3]:

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import Counter

nltk.download('punkt')
nltk.download('stopwords')

#stop_words = set(stopwords.words('english'))
#stemmer = PorterStemmer()

df_tok = df[:10000].copy() 

df_tok

[nltk_data] Downloading package punkt to /home/vcourrier/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vcourrier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
9995,"Fun, entertaining movie about WWII German spy ...",positive
9996,Give me a break. How can anyone say that this ...,negative
9997,This movie is a bad movie. But after watching ...,negative
9998,This is a movie that was probably made to ente...,negative


In [4]:
# Tokenize, remove stopwords, and stem
df_tok['review'] = df_tok['review'].apply(lambda x: word_tokenize(x.lower()))
print("Tokenized")
#df_tok['review'] = df_tok['review'].apply(lambda x: [word for word in x if word not in stop_words])
#print("Removed stopwords")
#df_tok['review'] = df_tok['review'].apply(lambda x: [stemmer.stem(word) for word in x])
#print("Stemmed")

Tokenized


In [5]:
# Limit vocabulary size
vocab_size = 10000
all_words = [word for review in df_tok['review'] for word in review]
word_counts = Counter(all_words)
most_common_words = set([word for word, count in word_counts.most_common(vocab_size)])

df_tok['review'] = df_tok['review'].apply(lambda x: [word if word in most_common_words else '<UNK>' for word in x])

# Encode reviews
word2value = {word: idx for idx, word in enumerate(most_common_words, start=1)}
word2value['<UNK>'] = 0

df_enc = df_tok.copy()
df_enc['review'] = df_enc['review'].apply(lambda x: [word2value[word] for word in x])

# Convert to tensors and pad
review_tensors = [torch.tensor(encoded_review) for encoded_review in df_enc['review']]
padded = pad_sequence(review_tensors, batch_first=True, padding_value=0).narrow(1, 0, 379)

In [6]:
# Initialize lists to store values for the DataFrame
reviews = []
sentiment_values = []

# Iterate over each row of the tensor
for i in range(padded.size(0)):
    # Extract the row from the tensor
    row = padded[i]
    
    # Convert the tensor row to a list and append it to the 'reviews' list
    reviews.append(row.tolist())
    # Extract the corresponding value of 'sentiment' column from the other DataFrame
    sentiment_value = df_tok.iloc[i]['sentiment']
    # Append the value to the 'sentiment_values' list
    if sentiment_value == "positive" :
        sentiment_values.append(1)
    else :
        sentiment_values.append(0)

# Final dataframe
final_df = pd.DataFrame({'review': reviews, 'sentiment': sentiment_values})

print(final_df)


####### Train-test split ########

# Split the data into training and testing sets (80% training, 20% testing)
train_df, test_df = train_test_split(final_df, test_size=0.15, random_state=42)

print("Training set shape:", train_df.shape)
print("Testing set shape:", test_df.shape)


class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Extract features and labels for a single row
        features = torch.tensor(self.dataframe.iloc[idx]['review'], dtype=torch.float32) 
        label = torch.tensor(self.dataframe.iloc[idx]['sentiment'], dtype=torch.long)  
        return features, label


                                                 review  sentiment
0     [1312, 2766, 1260, 7220, 6471, 9947, 8812, 282...          1
1     [4990, 4865, 6912, 9453, 1353, 5642, 8547, 447...          1
2     [8017, 9985, 9101, 2792, 4990, 4865, 1166, 450...          1
3     [9033, 2471, 5095, 4990, 6055, 7056, 4990, 691...          0
4     [0, 2433, 5095, 1139, 9470, 5914, 1260, 1526, ...          1
...                                                 ...        ...
9995  [7682, 6903, 7091, 2171, 3993, 9762, 4202, 425...          1
9996  [2023, 5609, 4990, 1708, 1353, 1421, 1667, 214...          0
9997  [9101, 2171, 1164, 4990, 5268, 2171, 1353, 576...          0
9998  [9101, 1164, 4990, 2171, 2829, 2792, 2236, 740...          0
9999  [6285, 1777, 3993, 3408, 1353, 6021, 1260, 676...          1

[10000 rows x 2 columns]
Training set shape: (8500, 2)
Testing set shape: (1500, 2)


In [7]:
# Define batch size and number of epochs
batch_size = 200

# Create custom datasets
train_dataset = CustomDataset(train_df)
test_dataset = CustomDataset(test_df)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


data_train = iter(train_loader)

In [8]:
len(train_dataset)/batch_size

42.5

In [ ]:
# Blocs du modèle

class View(nn.Module):
    def __init__(self, size):
        super(View, self).__init__()
        self.size = size

    def forward(self, tensor):
        return tensor.view(self.size)
    
class Encoder(nn.Module):
    def __init__(self, input_size, z_dim, hidden_dim, bidirectional=True):
        super(Encoder, self).__init__()
        self.z_dim = z_dim
        self.hidden_dim = hidden_dim
        self.hidden_factor = 2 if bidirectional else 1

        self.encoder = nn.LSTM(input_size, hidden_dim, bidirectional=True, batch_first=True)

        self.locs = nn.Linear(hidden_dim*self.hidden_factor, z_dim)
        self.scales = nn.Linear(hidden_dim*self.hidden_factor, z_dim)
    
    def forward(self, x):
        batch_size = x.size(0)
        # x should be of shape [batch_size, seq_len, input_size]
        if x.dim() == 1:
            x = x.unsqueeze(0).unsqueeze(0)  # shape [1, 1, input_size]
        elif x.dim() == 2:
            x = x.unsqueeze(1)  # shape [batch_size, 1, input_size]
        
        output, (hidden, c_n) = self.encoder(x)
        #print("pre hidden", h_n.shape)
        #hidden = h_n[-1]
        hidden = hidden.view(batch_size, self.hidden_dim*self.hidden_factor)

        locs = self.locs(hidden)
        scales = torch.clamp(F.softplus(self.scales(hidden)), min=1e-3)
        return locs, scales

class Decoder(nn.Module):
    def __init__(self, input_size, z_dim, hidden_dim, vocab_size, bidirectional=True):
        super(Decoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.input_size = input_size
        self.bidirectional = bidirectional
        self.hidden_factor = 2 if bidirectional else 1

        # Linear layer to transform z_dim to hidden_dim
        self.linear = nn.Linear(z_dim, hidden_dim * self.hidden_factor)

        # LSTM to generate sequences
        self.lstm = nn.LSTM(input_size, hidden_dim, bidirectional=bidirectional, batch_first=True)
        self.output = nn.Linear(hidden_dim * self.hidden_factor, vocab_size)

    def forward(self, z, xs):
        batch_size = z.size(0)
        seq_len = xs.size(1)

        # Transform latent vector to initial hidden state
        hidden = self.linear(z).view(self.hidden_factor, batch_size, self.hidden_dim)

        if xs.is_cuda:
            hidden = hidden.cuda()

        c_0 = torch.zeros(self.hidden_factor, batch_size, self.hidden_dim)
        if xs.is_cuda:
            c_0 = c_0.cuda()

        decoder_hidden = (hidden, c_0)

        # Forward pass through LSTM
        outputs, _ = self.lstm(xs, decoder_hidden)
        print('not crashed yet')

         # Print the shape of outputs before the linear layer
        print(f"Shape of LSTM outputs: {outputs.shape}")

        # Ensure the shape matches the expected input for the linear layer
        expected_shape = (batch_size, seq_len, self.hidden_dim * self.hidden_factor)
        if outputs.shape != expected_shape:
            print(f"Unexpected shape: {outputs.shape}, expected: {expected_shape}")
            return None  # or handle appropriately
        # Project each time step's output to the vocabulary size
        outputs = self.output(outputs)
        print('crashed?')
        # Apply log softmax to get log probabilities over the vocabulary
        logp = nn.functional.log_softmax(outputs, dim=-1)
        return logp

class Diagonal(nn.Module):
    def __init__(self, dim):
        super(Diagonal, self).__init__()
        self.dim = dim
        self.weight = nn.Parameter(torch.ones(self.dim))
        self.bias = nn.Parameter(torch.zeros(self.dim))

    def forward(self, x):
        return x * self.weight + self.bias

class Classifier(nn.Module):
    def __init__(self, dim):
        super(Classifier, self).__init__()
        self.dim = dim
        self.diag = Diagonal(self.dim)

    def forward(self, x):
        return self.diag(x)

class CondPrior(nn.Module):
    def __init__(self, dim):
        super(CondPrior, self).__init__()
        self.dim = dim
        self.diag_loc_true = nn.Parameter(torch.zeros(self.dim))
        self.diag_loc_false = nn.Parameter(torch.zeros(self.dim))
        self.diag_scale_true = nn.Parameter(torch.ones(self.dim))
        self.diag_scale_false = nn.Parameter(torch.ones(self.dim))

    def forward(self, x):
        x = x.unsqueeze(1) 
        loc = x * self.diag_loc_true + (1 - x) * self.diag_loc_false
        scale = x * self.diag_scale_true + (1 - x) * self.diag_scale_false
        return loc, torch.clamp(F.softplus(scale), min=1e-3)

In [10]:
# CCVAE model

def compute_kl(locs_q, scale_q, locs_p=None, scale_p=None):
    """
    Computes the KL(q||p)
    """
    if locs_p is None:
        locs_p = torch.zeros_like(locs_q)
    if scale_p is None:
        scale_p = torch.ones_like(scale_q)

    dist_q = dist.Normal(locs_q, scale_q)
    dist_p = dist.Normal(locs_p, scale_p)
    return dist.kl.kl_divergence(dist_q, dist_p).sum(dim=-1)

def img_log_likelihood(recon, xs):
        if xs.dim() == 1:
            xs = xs.unsqueeze(0).unsqueeze(0)  # shape [1, 1, input_size]
            recon = recon.unsqueeze(0).unsqueeze(0)
        elif xs.dim() == 2:
            xs = xs.unsqueeze(1)  # shape [batch_size, 1, input_size]
            recon = recon.unsqueeze(1)
        laplace_dist = dist.Laplace(recon, torch.ones_like(recon))
        log_prob = laplace_dist.log_prob(xs)
        return log_prob.sum(dim=(0, 1, 2))

def sentence_log_likelihood(recon, xs):
    if xs.dim() == 2:
        xs = xs.unsqueeze(1)  
        recon = recon.unsqueeze(1)

    normal_dist = dist.Normal(recon, torch.ones_like(recon))  
    log_prob = normal_dist.log_prob(xs)
    return log_prob.sum(dim=(1, 2))

class CCVAE(nn.Module):
    """
    CCVAE
    """
    def __init__(self, input_size, hidden_dim, vocab_size, z_dim, num_classes, use_cuda, prior_fn):
        super(CCVAE, self).__init__()
        self.z_dim = z_dim
        self.input_size = input_size
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.z_classify = num_classes
        self.z_style = z_dim - num_classes
        self.use_cuda = use_cuda
        self.num_classes = num_classes
        self.ones = torch.ones(1, self.z_style)
        self.zeros = torch.zeros(1, self.z_style)
        self.y_prior_params = prior_fn

        self.classifier = Classifier(self.num_classes)

        self.encoder = Encoder(self.input_size, self.z_dim, self.hidden_dim)
        self.decoder = Decoder(self.input_size, self.z_dim, self.hidden_dim, self.vocab_size)

        self.cond_prior = CondPrior(self.num_classes)

        if self.use_cuda:
            self.ones = self.ones.cuda()
            self.zeros = self.zeros.cuda()
            self.y_prior_params = self.y_prior_params.cuda()
            self.cuda()

    def sup(self, x, y):
        y = y.float()
        bs = x.shape[0]
        #inference
        post_params = self.encoder(x)
        
        z = dist.Normal(*post_params).rsample()
        zc, zs = z.split([self.z_classify, self.z_style], 1)
        qyzc = dist.Bernoulli(logits=self.classifier(zc))
        log_qyzc = qyzc.log_prob(y).sum(dim=-1)

        # compute kl
        locs_p_zc, scales_p_zc = self.cond_prior(y)
        prior_params = (torch.cat([locs_p_zc, self.zeros.expand(bs, -1)], dim=1), 
                        torch.cat([scales_p_zc, self.ones.expand(bs, -1)], dim=1))
        kl = compute_kl(*post_params, *prior_params)

        #compute log probs for x and y

        log_py = dist.Bernoulli(self.y_prior_params.expand(bs, -1)).log_prob(y).sum(dim=-1)
        print('not crashed')
        recon = self.decoder(z, x)
        print('crashed?')
        

        print('here?')
        log_qyx = self.classifier_loss(x, y)
        #recon = recon.transpose(0, 1)
        #recon = recon.reshape(bs, -1, 1)  # Transpose back to [batch_size, seq_len, hidden_dim]
        #log_pxz = sentence_log_likelihood(recon, x)
        
        NLL = nn.NLLLoss(ignore_index=0, reduction='sum')  # Assuming 0 is the padding index
        log_pxz = NLL(recon.view(-1, self.vocab_size), y.view(-1))
        print('crashed')

        # we only want gradients wrt to params of qyz, so stop them propogating to qzx
        log_qyzc_ = dist.Bernoulli(logits=self.classifier(zc.detach())).log_prob(y).sum(dim=-1)
        w = torch.exp(log_qyzc_ - log_qyx)+ 1e-8
        elbo = (w * (log_pxz - kl - log_qyzc) + log_py + log_qyx).mean()
        return -elbo

    def classifier_loss(self, x, y, k=100):
        """
        Computes the classifier loss.
        """
        zc, _ = dist.Normal(*self.encoder(x)).rsample(torch.tensor([k])).split([self.z_classify, self.z_style], -1)
        logits = self.classifier(zc.view(-1, self.z_classify))
        d = dist.Bernoulli(logits=logits)
        y = y.unsqueeze(0).unsqueeze(-1)
        y = y.expand(k, -1, -1).contiguous().view(-1, self.num_classes)
        lqy_z = d.log_prob(y).view(k, x.shape[0], self.num_classes).sum(dim=-1)
        lqy_x = torch.logsumexp(lqy_z, dim=0) - np.log(k)
        return lqy_x

    #def reconstruct_img(self, x):
    #    return self.decoder(dist.Normal(*self.encoder(x)).rsample())

    def classifier_acc(self, x, y=None, k=1):
        zc, _ = dist.Normal(*self.encoder(x)).rsample(torch.tensor([k])).split([self.z_classify, self.z_style], -1)
        logits = self.classifier(zc.view(-1, self.z_classify)).view(-1, self.num_classes)
        y = y.unsqueeze(0).unsqueeze(-1)
        y = y.expand(k, -1, -1).contiguous().view(-1, self.num_classes)
        preds = torch.round(torch.sigmoid(logits))
        acc = (preds.eq(y)).float().mean()
        return acc
    
    def accuracy(self, data_loader, *args, **kwargs):
        acc = 0.0
        for (x, y) in data_loader:
            if self.use_cuda:
                x, y = x.cuda(), y.cuda()
            batch_acc = self.classifier_acc(x, y)
            acc += batch_acc
        return acc / len(data_loader)
 

In [11]:
input_size = 1 
seq_len = 379
hidden_dim = 256
z_dim = 28
num_classes = 1
use_cuda = False
prior_fn = torch.tensor([[0.5]])
n_epochs = 10

In [12]:
num_batch = int(len(train_dataset)/batch_size)
num_batch

42

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)

: 

In [ ]:
import gc

# Training
cc_vae = CCVAE(input_size, hidden_dim, vocab_size, z_dim, num_classes, use_cuda, prior_fn)

cc_vae.apply(init_weights)

optim = torch.optim.Adam(params=cc_vae.parameters(), lr=0.001)

for epoch in range(n_epochs):
    print("Epoch:", epoch)
    cc_vae.train()
    
    epoch_losses_sup = 0.0
    #for i in tqdm(range(num_batch)):
    for batch_idx, (xs, ys) in enumerate(train_loader):
        print(batch_idx)
        xs = xs.view(xs.shape[0], seq_len, input_size)
        loss = cc_vae.sup(xs, ys)
        
        optim.zero_grad() 
        loss.backward()
        optim.step()

        epoch_losses_sup += loss.detach().item()
        print(f"Batch {batch_idx} Loss: {loss.detach().item():.3f}")
        gc.collect()
    
    avg_loss = epoch_losses_sup / len(train_loader)
    print(f"[Epoch {epoch+1:03d}] Sup Loss: {avg_loss:.3f}")

cc_vae.eval() 
test_acc = cc_vae.accuracy(test_loader)

print(test_acc)

Epoch: 0
0
not crashed
not crashed yet
Shape of LSTM outputs: torch.Size([200, 379, 512])
